<a href="https://colab.research.google.com/github/mehluli92/Sentiment_Analysis_Movie_Reviews/blob/main/Sentiment_Analysis_IMBD_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import files
uploaded = files.upload()


# New section

In [ ]:
data = pd.read_csv("IMDB_Dataset.csv")
data.head()

In [ ]:
data.shape

In [ ]:
type(data)

In [ ]:
data.tail()

In [ ]:
data["sentiment"].value_counts()

In [ ]:
# label encording positive = 1, negetive = 0
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)
data.head()

In [ ]:
# Use LSTM -> Long Short Term Memory
# RNN -> not good textual data because it has no memory
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
train_data, test_data = train_test_split(data, test_size = 0.2, random_state=42)

In [ ]:
train_data.shape

In [ ]:
test_data.shape

In [ ]:
tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(train_data["review"])

In [ ]:
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [ ]:
X_train

In [ ]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [ ]:
Y_train

In [ ]:
# LSTM Model Building

In [ ]:
model = Sequential()
model.add(Embedding(input_dim = 5000, output_dim = 128, input_shape=(200,)))
model.add(LSTM(128, dropout=0.2,recurrent_dropout = 0.2 ))
model.add(Dense(1, activation = "sigmoid"))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = "adam", loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
model.fit(X_train, Y_train, epochs = 5, batch_size = 64, validation_split = 0.2 )

In [ ]:
model.save("model.h5")

In [ ]:
import joblib
joblib.dump(tokenizer, "tokenizer.pkl")

In [ ]:
loss, accuracy = model.evaluate(X_test, Y_test)

In [ ]:
# Building Predictive System

In [ ]:
def predictive_system(review):
  sequences = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequences, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [ ]:
predictive_system("This movies was amazing")

In [ ]:
predictive_system("A thrilling adventure with stunning visual")